# Imports & Setup

In [1]:
from typing import List

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import chi2, uniform
import statsmodels.api as sm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import  FunctionTransformer, OneHotEncoder, PolynomialFeatures, StandardScaler

In [2]:
# Enable diagrams to visualize pipelines
from sklearn import set_config
set_config(display="diagram")

# Functions and Classes

In [3]:
def split_bmi_in_three(x: float) -> str:
    if x < 25:
        return "underweight_normal"
    if x < 30:
        return "overweight"
    return "obesity"

In [17]:
class ThresholdBinningTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column: str, bins: List[float], labels=List[str]):
        self.column = column
        self.bins = bins
        self.labels = labels

    def fit(self, X, y=None):
        # No fitting necessary for this transformer
        return self

    def transform(self, X):
        if self.column in X.columns:
            X_binned = pd.cut(X[self.column],
                              bins=self.bins, labels=self.labels,
                              right=False)  # left edge inclusive, right edge exclusive
            X_transformed = X.copy()
            X_transformed[self.column] = X_binned
            return X_transformed
        else:
            raise ValueError(f"Column {self.column} not in input") 

**Each following suggestion gives a different TraceBack...**

In [5]:
# def apply_bmi_split(X: np.array) -> np.array:
#     X[:, 2] = np.apply_along_axis(split_bmi_in_three, 1, X[: 2])
#     return X

In [3]:
# def apply_bmi_split(X: np.array) -> np.array:
#     return (np
#             .apply_along_axis(split_bmi_in_three, 1, X[: 2])
#             .reshape(-1, 1)
#            )

In [4]:
# def apply_bmi_split(column):
#     return column.apply(split_bmi_in_three)

In [2]:
# def apply_bmi_split(column):
#     return np.array([
#         split_bmi_in_three(float(x)) for x in column
#     ])

In [14]:
# def apply_bmi_split(column):
#     new_column = []
#     for elem in column:
#         if elem < 25:
#             new_column.append("underweight_normal")
#         elif elem < 30:
#             new_column.append("overweight")
#         else:
#             new_column.append("obestity")
#     return np.array(new_column)

In [5]:
def apply_bmi_split(column):
    new_column = []
    for elem in column:
        if isinstance(elem, str):
            new_column.append(elem)
        elif elem < 25:
            new_column.append("underweight_normal")
        elif elem < 30:
            new_column.append("overweight")
        else:
            new_column.append("obestity")
    return np.array(new_column).reshape(-1, 1)

# Data Loading & Separating Features / Target

In [6]:
df = pd.read_csv("csvs/cleaned_dataset.csv")

In [7]:
y = df.pop("charges")
X = df

### Modifying `y`'s shape

In [8]:
y = np.log(y + 1)

# Preprocessing

## With Binning `bmi` Inside PipeLine

### Hold-Out

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    shuffle=True,
                                                    train_size=0.85,
                                                    random_state=42,
                                                    stratify=X['smoker'])

### Pipeline

In [18]:
bmi_edges = [0.0, 25.0, 30.0, np.inf]
bmi_cats = ["underweight_normal", "overweight", "obesity"]

bmi_categorizer = ThresholdBinningTransformer(column="bmi", bins=bmi_edges, labels=bmi_cats)    

In [19]:
ohe_nom = OneHotEncoder(drop="first", handle_unknown="ignore")
ohe_bin = OneHotEncoder(drop="if_binary", handle_unknown="ignore")
poly = PolynomialFeatures(degree=2)
std = StandardScaler()

In [20]:
en = ElasticNet(random_state=42, 
                max_iter=10_000, tol=1e-3
)

In [21]:
pipe_bmi = make_pipeline(bmi_categorizer, ohe_nom)
pipe_bmi

Pipeline(steps=[('thresholdbinningtransformer',
                 ThresholdBinningTransformer(bins=[0.0, 25.0, 30.0, inf],
                                             column='bmi',
                                             labels=['underweight_normal',
                                                     'overweight',
                                                     'obesity'])),
                ('onehotencoder',
                 OneHotEncoder(drop='first', handle_unknown='ignore'))])

In [22]:
encoder_1 = ColumnTransformer(
    transformers = [
        ("bmi", pipe_bmi, ["bmi"]),
        ("bin", ohe_bin, ["sex", "smoker"]),
        ("ohe", ohe_nom, ["region"])
    ],
    remainder="passthrough")

encoder_1

ColumnTransformer(remainder='passthrough',
                  transformers=[('bmi',
                                 Pipeline(steps=[('thresholdbinningtransformer',
                                                  ThresholdBinningTransformer(bins=[0.0,
                                                                                    25.0,
                                                                                    30.0,
                                                                                    inf],
                                                                              column='bmi',
                                                                              labels=['underweight_normal',
                                                                                      'overweight',
                                                                                      'obesity'])),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['bmi']),
                                ('bin',
                                 OneHotEncoder(drop='if_binary',
                                               handle_unknown='ignore'),
                                 ['sex', 'smoker']),
                                ('ohe',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['region'])])

In [23]:
model_1 = make_pipeline(encoder_1, poly, std, en)
model_1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bmi',
                                                  Pipeline(steps=[('thresholdbinningtransformer',
                                                                   ThresholdBinningTransformer(bins=[0.0,
                                                                                                     25.0,
                                                                                                     30.0,
                                                                                                     inf],
                                                                                               column='bmi',
                                                                                               labels=['underweight_normal',
                                                                                                       'overweight',
                                                                                                       'obesity'])),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['bmi']),
                                                 ('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(max_iter=10000, random_state=42, tol=0.001))])

### Training & Score

In [24]:
%time

params = {
    "elasticnet__alpha": uniform(0, 2),
    "elasticnet__l1_ratio": uniform(0, 1)
}

random_search_1 = RandomizedSearchCV(
    model_1,
    param_distributions=params,
    n_iter=2_000,
    cv=10,
    n_jobs=-1,
    random_state=42
)

random_search_1.fit(X_train, y_train)

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.81 µs


/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.850e+00, tolerance: 8.441e-01
  model = cd_fast.enet_coordinate_descent(
/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.941e+00, tolerance: 8.600e-01
  model = cd_fast.enet_coordinate_descent(
/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the featu

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('bmi',
                                                                               Pipeline(steps=[('thresholdbinningtransformer',
                                                                                                ThresholdBinningTransformer(bins=[0.0,
                                                                                                                                  25.0,
                                                                                                                                  30.0,
                                                                                                                                  inf],
                                                                                                                            column='bmi',
                                                                                                                            labels=['underweight_normal',
                                                                                                                                    'overweight',
                                                                                                                                    'obesity'])),
                                                                                               ('onehotencoder',
                                                                                                OneHotEncoder(drop='first',
                                                                                                              han...
                                             ('standardscaler',
                                              StandardScaler()),
                                             ('elasticnet',
                                              ElasticNet(max_iter=10000,
                                                         random_state=42,
                                                         tol=0.001))]),
                   n_iter=2000, n_jobs=-1,
                   param_distributions={'elasticnet__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f0056118290>,
                                        'elasticnet__l1_ratio': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f0055f04850>},
                   random_state=42)

In [25]:
best_model_1 = random_search_1.best_estimator_
best_model_1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bmi',
                                                  Pipeline(steps=[('thresholdbinningtransformer',
                                                                   ThresholdBinningTransformer(bins=[0.0,
                                                                                                     25.0,
                                                                                                     30.0,
                                                                                                     inf],
                                                                                               column='bmi',
                                                                                               labels=['underweight_normal',
                                                                                                       'overweight',
                                                                                                       'obesity'])),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['bmi']),
                                                 (...
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.00026938600897064013,
                            l1_ratio=0.5111291392372298, max_iter=10000,
                            random_state=42, tol=0.001))])

In [27]:
best_model_1.fit(X_train, y_train)
best_model_1.score(X_test, y_test)

0.9177282544723908

## With Binning `bmi` Outside Pipeline

In [15]:
X_bmi_nom = X.copy()

In [16]:
X_bmi_nom.bmi = X_bmi_nom.bmi.apply(split_bmi_in_three)

### Hold-Out

In [17]:
X_bmi_nom_train, X_bmi_nom_test, y_train, y_test =\
train_test_split(X_bmi_nom, y,
                 shuffle=True,
                 train_size=0.85,
                 random_state=42,
                 stratify=X['smoker'])

### Pipeline

In [18]:
encoder_2 = ColumnTransformer(
    transformers=[
        ("bin", ohe_bin, ["sex", "smoker"]),
        ("nom", ohe_nom, ["bmi", "region"])
    ],
    remainder="passthrough"
)
encoder_2

ColumnTransformer(remainder='passthrough',
                  transformers=[('bin',
                                 OneHotEncoder(drop='if_binary',
                                               handle_unknown='ignore'),
                                 ['sex', 'smoker']),
                                ('nom',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore'),
                                 ['bmi', 'region'])])

In [20]:
model_2 = make_pipeline(encoder_2, poly, std, en)
model_2

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('nom',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['bmi', 'region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(max_iter=10000, random_state=42, tol=0.001))])

### Training & Score

In [24]:
%%time

params = {
    "elasticnet__alpha": uniform(0, 2),
    "elasticnet__l1_ratio": uniform(0, 1)
}

random_search_2 = RandomizedSearchCV(
    model_2,
    param_distributions=params,
    n_iter=2_000,
    cv=10,
    n_jobs=-1,
    random_state=42
)

random_search_2.fit(X_bmi_nom_train, y_train)

/home/martin/Documents/PolicyPriceAI/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.980e+00, tolerance: 8.600e-01
  model = cd_fast.enet_coordinate_descent(
/home/martin/Documents/PolicyPriceAI/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.718e+00, tolerance: 8.441e-01
  model = cd_fast.enet_coordinate_descent(
/home/martin/Documents/PolicyPriceAI/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of 

CPU times: user 35.2 s, sys: 829 ms, total: 36 s
Wall time: 1min 17s


RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('bin',
                                                                               OneHotEncoder(drop='if_binary',
                                                                                             handle_unknown='ignore'),
                                                                               ['sex',
                                                                                'smoker']),
                                                                              ('nom',
                                                                               OneHotEncoder(drop='first',
                                                                                             handle_unknown='ignore'),
                                                                               ['bmi',
                                                                                'region'])])),
                                             ('polynomialfeatures',
                                              PolynomialFeatures()),
                                             ('standardscaler',
                                              StandardScaler()),
                                             ('elasticnet',
                                              ElasticNet(max_iter=10000,
                                                         random_state=42,
                                                         tol=0.001))]),
                   n_iter=2000, n_jobs=-1,
                   param_distributions={'elasticnet__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f150879dcc0>,
                                        'elasticnet__l1_ratio': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f1501ee52a0>},
                   random_state=42)

In [25]:
best_model_2 = random_search_2.best_estimator_
best_model_2

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('nom',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['bmi', 'region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.00026938600897064013,
                            l1_ratio=0.5111291392372298, max_iter=10000,
                            random_state=42, tol=0.001))])

In [26]:
best_model_2.fit(X_bmi_nom_train, y_train)
best_model_2.score(X_bmi_nom_test, y_test)

0.9177335980741432

# 💿 Save model

In [28]:
joblib.dump(best_model_1, "model.joblib")

['model.joblib']

# Cook's Distance

In [29]:
X_train_preproc = best_model_1[:-1].fit_transform(X_train)
X_train_preproc.shape

(1136, 55)

## With `statsmodels`

In [30]:
sm_model = sm.OLS(y_train, sm.add_constant(X_train_preproc)).fit()

In [32]:
influence = sm_model.get_influence()
cook_distance = influence.cooks_distance[0]

In [35]:
n, p = X_train_preproc.shape

In [36]:
cook_threshold = 4 / (n - p)
cook_threshold

0.0037002775208140612

In [37]:
(cook_distance > cook_threshold).sum()

47

<font color="orangered">**There are 47 influent values. Let's retrieve their indexes.**</font>

### Retrieving Indexes

In [38]:
condition = cook_distance > cook_threshold

In [39]:
indexes = np.where(condition)
indexes

(array([  15,   33,   45,   82,   93,  105,  130,  170,  193,  199,  301,
         320,  330,  387,  412,  418,  445,  466,  481,  488,  515,  536,
         554,  568,  642,  700,  706,  715,  716,  779,  794,  798,  802,
         821,  847,  893,  906,  931,  946,  987, 1039, 1055, 1077, 1092,
        1100, 1119, 1124]),)

### Retrieving Records

In [40]:
df = pd.read_csv("csvs/cleaned_dataset.csv")
df_influents = df.iloc[indexes]
df_influents

,age,sex,bmi,children,smoker,region,charges
15,19,male,24.600,1,no,southwest,1837.23700
33,63,male,28.310,0,no,northwest,13770.09790
45,55,male,37.300,0,no,southwest,20630.28351
82,22,male,37.620,1,yes,southeast,37165.16380
93,35,male,34.770,2,no,northwest,5729.00530
105,20,male,28.025,1,yes,northwest,17560.37975
130,59,female,26.505,0,no,northeast,12815.44495
170,63,male,41.470,0,no,southeast,13405.39030
193,56,female,26.600,1,no,northwest,12044.34200
199,64,female,39.330,0,no,northeast,14901.51670


### 💿 Exporting Influents

In [41]:
df_influents.to_csv("csvs/influents.csv")

## With `sklearn`

In [44]:
sl_model = LinearRegression().fit(X_train_preproc, y_train)

In [46]:
n, p = X_train_preproc.shape

In [97]:
X_tp = X_train_preproc.copy()

In [98]:
X_tp.shape

(1136, 55)

In [99]:
# Reduced Train Sets: X_red, y_red
# X_reds[i] is X, y without the ith row
X_y_reds = []
for idx in range(len(X_tp)):
    X_y_reds.append((np.delete(X, i, axis=0), np.delete(y_train, i, axis=0)))

In [100]:
X_reds[0].shape

(1135, 55)

In [101]:
betas_reds = []
for X_red, y_red in X_y_reds:
    model = LinearRegression().fit(X_red, y_red)
    betas_reds.append(model.coef_)  

In [102]:
beta = LinearRegression().fit(X, y_train).coef_

In [104]:
beta.shape

(55,)

In [105]:
deltas_beta = [beta - beta_red for beta_red in betas_reds]

In [124]:
all(deltas_beta[0] == deltas_beta[2])

True

In [107]:
len(deltas_beta)

1136

In [108]:
residuals = y_train - sl_model.predict(X_train_preproc)

In [109]:
n, p = X_train_preproc.shape

In [110]:
residuals ** 2

1094    0.166202
94      0.033765
349     0.057924
410     0.080895
981     0.048910
          ...   
674     0.001376
752     0.054482
319     0.048461
934     0.013979
1113    0.025091
Name: charges, Length: 1136, dtype: float64

In [111]:
residuals_variance = 1 / (n - p) * np.sum(residuals ** 2)

In [113]:
mat = X_tp.T @ X_tp

In [114]:
mat.shape

(55, 55)

In [121]:
np.round(mat)

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0., 1136.,   62., ...,  -26.,  -21.,   -5.],
       [   0.,   62., 1136., ...,  -46.,    7.,  -11.],
       ...,
       [   0.,  -26.,  -46., ..., 1136.,  230.,  -31.],
       [   0.,  -21.,    7., ...,  230., 1136.,  947.],
       [   0.,   -5.,  -11., ...,  -31.,  947., 1136.]])

In [115]:
cooks = []
for db in deltas_beta:
    cooks.append(1 / p * (db.T @ mat @ db) / residuals_variance)

In [116]:
cooks

[0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.36666561469684356,
 0.3666656

In [144]:
4 / (n - p)

0.0037002775208140612

# 💿 Removing Influent Outliers 

In [49]:
df_std = df.iloc[list(set(df.index) - set(indexes[0]))]
df_std

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [43]:
df_std.to_csv("csvs/standard.csv")

# Training Again on *Inliers*

In [50]:
y_std = df_std.pop("charges")
X_std = df_std

y_std = np.log(y_std + 1)

In [51]:
X_std_train, X_std_test, y_std_train, y_std_test = train_test_split(
    X_std, y_std,
    shuffle=True,
    train_size=0.85,
    random_state=42,
    stratify=X_std['smoker']
)

In [52]:
model_1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bmi',
                                                  Pipeline(steps=[('thresholdbinningtransformer',
                                                                   ThresholdBinningTransformer(bins=[0.0,
                                                                                                     25.0,
                                                                                                     30.0,
                                                                                                     inf],
                                                                                               column='bmi',
                                                                                               labels=['underweight_normal',
                                                                                                       'overweight',
                                                                                                       'obesity'])),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['bmi']),
                                                 ('bin',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(max_iter=10000, random_state=42, tol=0.001))])

In [53]:
%time

params = {
    "elasticnet__alpha": uniform(0, 2),
    "elasticnet__l1_ratio": uniform(0, 1)
}

random_search_std = RandomizedSearchCV(
    model_1,
    param_distributions=params,
    n_iter=2_000,
    cv=10,
    n_jobs=-1,
    random_state=42
)

random_search_std.fit(X_std_train, y_std_train)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs


/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.492e+00, tolerance: 8.515e-01
  model = cd_fast.enet_coordinate_descent(
/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.017e+00, tolerance: 8.377e-01
  model = cd_fast.enet_coordinate_descent(
/home/greg/projects/PolicyPriceAI/venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the featu

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('bmi',
                                                                               Pipeline(steps=[('thresholdbinningtransformer',
                                                                                                ThresholdBinningTransformer(bins=[0.0,
                                                                                                                                  25.0,
                                                                                                                                  30.0,
                                                                                                                                  inf],
                                                                                                                            column='bmi',
                                                                                                                            labels=['underweight_normal',
                                                                                                                                    'overweight',
                                                                                                                                    'obesity'])),
                                                                                               ('onehotencoder',
                                                                                                OneHotEncoder(drop='first',
                                                                                                              han...
                                             ('standardscaler',
                                              StandardScaler()),
                                             ('elasticnet',
                                              ElasticNet(max_iter=10000,
                                                         random_state=42,
                                                         tol=0.001))]),
                   n_iter=2000, n_jobs=-1,
                   param_distributions={'elasticnet__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f0047662150>,
                                        'elasticnet__l1_ratio': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f0046b30e90>},
                   random_state=42)

In [54]:
best_model_std = random_search_std.best_estimator_
best_model_std

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('bmi',
                                                  Pipeline(steps=[('thresholdbinningtransformer',
                                                                   ThresholdBinningTransformer(bins=[0.0,
                                                                                                     25.0,
                                                                                                     30.0,
                                                                                                     inf],
                                                                                               column='bmi',
                                                                                               labels=['underweight_normal',
                                                                                                       'overweight',
                                                                                                       'obesity'])),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  ['bmi']),
                                                 (...
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'),
                                                  ['sex', 'smoker']),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['region'])])),
                ('polynomialfeatures', PolynomialFeatures()),
                ('standardscaler', StandardScaler()),
                ('elasticnet',
                 ElasticNet(alpha=0.0013067815225991097,
                            l1_ratio=0.8706690341255177, max_iter=10000,
                            random_state=42, tol=0.001))])

In [55]:
best_model_std.fit(X_std_train, y_std_train)
best_model_std.score(X_std_test, y_std_test)

0.8744644131171115

❗ <font color="orangered">**Unfortunately, the score decreased on the full set of inliers.**</font>